### 一、研究动力

1、新增了许多用户行为流稀疏特征，共276维，希望能通过embedding将其抽象成dense的特征，更进一步的话，希望可解释

2、因为模型训练的要求，需要dump线上每个商品的全部特征，考虑性能的因素，将稀疏特征压缩是很有必要的

### 二、现实场景

如下图所示,原始稀疏特征是:

``` c++
allcid_brand_top_behav_type     // 若nan的话，赋值为-1
allcid_brand_cnt  
allcid_brand_type1_cnt
allcid_brand_type0_cnt
allcid_brand_prob            //其实最原始的应该将allcid_brand_prob替换成sum,但是目前数据就是如此，除非改线上代码，重新dump
allcid_brand_top_behav_type_time_diff   // 若为nan的话，赋值为时间窗口的最大值,比如时间窗口是3天，那么就赋值为3*24*60*60
allcid_brand_last_behav_type_time_diff  // 若为nan的话，赋值为时间窗口的最大值,比如时间窗口是3天，那么就赋值为3*24*60*60
```

人工通过熵、互信息等统计量含义将这些特征组合后得到的新特征，依旧是稀疏特征

![1](https://github.com/yanqili/photos/raw/master/1.PNG)

### 三、阅读相关文献

#### 1、Google在2016年提出的[Wide & Deep Learning for Recommender Systems](https://arxiv.org/abs/1606.07792)

以及美团基于这一思想上的实践[深度学习在美团点评推荐平台排序中的运用](https://mp.weixin.qq.com/s/847h4ITQMtUlZcurJ9Vlvg?scene=25##)

总的来说，用wide框架，即简单的LR模型来学习用户的历史行为；用deep框架，即神经网络，可以来泛化用户的喜好。

**那么为什么要这么做呢？**
> 个人理解

>因为，现在用到的特征中，有大量的稀疏特征，如果只用wide框架，想要达到泛化的能力，就必须人工做很多特征工程。如果只用deep框架，泛化能力是有了，但是有些特征本身能明显区分用户行为，但是经过神经网络后，这种区分能力降低了。

>文章中是这么说的：

>Embedding-based models, such as factorization machines
or deep neural networks, can generalize to previously unseen query-item feature pairs by learning a low-dimensional
dense embedding vector for each query and item feature,
with less burden of feature engineering. However, it is difficult to learn effective low-dimensional representations for
queries and items when the underlying query-item matrix is
sparse and high-rank, such as users with specific preferences
or niche items with a narrow appeal. In such cases, there
should be no interactions between most query-item pairs,
but dense embeddings will lead to nonzero predictions for all
query-item pairs, and thus can over-generalize and make less
relevant recommendations. On the other hand, linear models with cross-product feature transformations can memorize
these "exception rules" with much fewer parameters

2篇文章相比而言，还是美团的更贴地气一些，有更为具体是实现过程，如下图

google结构

![](http://img.blog.csdn.net/20170502135611349)

美团结构

![](http://mmbiz.qpic.cn/mmbiz_png/hEx03cFgUsUA2chtX4yXmEXpoH6epAapKrHlSLqWCk5vHic0oGEYwPg65CIUaZphQC20xicQEY7tYD6ibOtpsvkvg/0?wx_fmt=png)

这里存在疑惑的是，美团文章中写的与他们的图示结构不一致：
> 在训练期间，与Wide & Deep Learning论文中不同之处在于，我们将组合特征作为输入层分别输入到对应的Deep组件和Wide组件中。然后在Deep部分将全部输入数据送到3个ReLU层，在最后通过Sigmoid层进行打分。

我的理解是，应该将连续特征、组合特征、离散特征都同时输入到Deep组件和wide组件中

在我们的现实场景中，造的一些类目下品牌偏好、价格偏好的特征都是属于这里的组合特征

** 这里发散一下思维，Deep组件里第一层是embedding层，感觉就是做的降维的事，SVD也可以做这事，比如PCA降维，但是呢，是有区别的。**

这里是通过神经网络，在降低损失函数值的时候，得到输入层与embedding层之间的权重，也就是转换矩阵。而PCA降维是非监督学习的降维方法。

#### 2、微软2016的一篇文章[Deep Crossing: Web-Scale Modeling without Manually Crafted Combinatorial Features](https://www.kdd.org/kdd2016/papers/files/adf0975-shanA.pdf)

同时参考一篇国人的讲解[wide & Deep 和 Deep & Cross](https://paper.tuisec.win/detail/0c7ccb678258065)

![](https://cdn.tuisec.win/full/upload/201801/70f121fda37852f17acfe9bd8e99863a.jpg)

这篇的感觉就是，组合特征也不用自己做了，用cross网络自己去组合、筛选，但是解释性应该就没有人工组合的那么强了

#### 3、[《Entity Embeddings of Categorical Variables》](https://arxiv.org/pdf/1604.06737.pdf)

> Embedding的起源和火爆都是在NLP中的，经典的word2vec都是在做word embedding这件事情，而真正首先在结构数据探索embedding的是在kaggle上的《Rossmann Store Sales》中的rank 3的解决方案，作者在比赛完后为此方法整理一篇论文放在了arXiv，文章名：《Entity Embeddings of Categorical Variables》。

这篇文章尚未仔细看，待有时间再看